In [1]:
import os
from glob import glob
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning) 

import numpy as np
# import matplotlib.pyplot as plt

import mne
mne.set_log_level(verbose=False)
import wandb
from tqdm.auto import tqdm

from DatasetsPaco import *

In [2]:
dataset1 = EDFData("../Data/PSG1.edf", channels=['F4'])

In [4]:
data1 = dataset1.epochs.get_data()
data1.shape

(882, 1, 15360)

In [5]:
data1 = np.squeeze(data1, axis=1)
data1.shape

(882, 15360)

In [8]:
labels = dataset1.epochs.events[:,-1]
labels.shape

(882,)

In [12]:
dataset1.epochs.event_id

{'Sleep stage N1': 1,
 'Sleep stage N2': 2,
 'Sleep stage N3': 3,
 'Sleep stage R': 4,
 'Sleep stage W': 5}

In [14]:
labels_binary = []

for label in labels:
    if label == 5:
        labels_binary.append(1) # Awake -> 1
    else:
        labels_binary.append(0) # Sleeping -> 0

labels_binary = np.array(labels_binary)
labels_binary.shape

(882,)

In [2]:
def create_simple_model():
    model = tf.keras.models.Sequential([
        tf.keras.layers.Conv1D(128, 512//2, strides=512//4, input_shape=(15360, 1), activation="relu"),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(1, activation="sigmoid")
    ])
    model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
    return model

In [24]:
model = create_simple_model()
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_2 (Conv1D)            (None, 119, 128)          32896     
_________________________________________________________________
flatten_2 (Flatten)          (None, 15232)             0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 15233     
Total params: 48,129
Trainable params: 48,129
Non-trainable params: 0
_________________________________________________________________


In [20]:
history = model.fit(np.expand_dims(data1,-1), labels_binary, batch_size=16, epochs=10)

Epoch 1/10
56/56 [==============================] - 18s 316ms/step - loss: 0.6121 - accuracy: 0.7177
Epoch 2/10
56/56 [==============================] - 0s 5ms/step - loss: 0.5945 - accuracy: 0.7211
Epoch 3/10
56/56 [==============================] - 0s 5ms/step - loss: 0.5959 - accuracy: 0.7211
Epoch 4/10
56/56 [==============================] - 0s 5ms/step - loss: 0.5974 - accuracy: 0.7211
Epoch 5/10
56/56 [==============================] - 0s 5ms/step - loss: 0.5997 - accuracy: 0.7211
Epoch 6/10
56/56 [==============================] - 0s 6ms/step - loss: 0.5944 - accuracy: 0.7211
Epoch 7/10
56/56 [==============================] - 0s 5ms/step - loss: 0.5952 - accuracy: 0.7211
Epoch 8/10
56/56 [==============================] - 0s 6ms/step - loss: 0.5983 - accuracy: 0.7211
Epoch 9/10
56/56 [==============================] - 0s 6ms/step - loss: 0.5972 - accuracy: 0.7211
Epoch 10/10
56/56 [==============================] - 0s 6ms/step - loss: 0.5989 - accuracy: 0.7211


In [21]:
mean = data1.mean()
std = data1.std()
mean, std

(2.941610868569235e-05, 0.0001535793839162135)

In [22]:
data1_std = (data1 - mean)/std

In [25]:
model = create_simple_model()
history = model.fit(np.expand_dims(data1_std,-1), labels_binary, batch_size=16, epochs=10)

Epoch 1/10
56/56 [==============================] - 0s 8ms/step - loss: 0.7652 - accuracy: 0.7177
Epoch 2/10
56/56 [==============================] - 0s 5ms/step - loss: 0.6117 - accuracy: 0.7041
Epoch 3/10
56/56 [==============================] - 0s 5ms/step - loss: 0.5421 - accuracy: 0.7551
Epoch 4/10
56/56 [==============================] - 0s 5ms/step - loss: 0.5147 - accuracy: 0.7506
Epoch 5/10
56/56 [==============================] - 0s 5ms/step - loss: 0.4962 - accuracy: 0.7551
Epoch 6/10
56/56 [==============================] - 0s 5ms/step - loss: 0.4782 - accuracy: 0.7664
Epoch 7/10
56/56 [==============================] - 0s 5ms/step - loss: 0.4555 - accuracy: 0.7823
Epoch 8/10
56/56 [==============================] - 0s 6ms/step - loss: 0.4298 - accuracy: 0.7891
Epoch 9/10
56/56 [==============================] - 0s 6ms/step - loss: 0.4103 - accuracy: 0.8005
Epoch 10/10
56/56 [==============================] - 0s 5ms/step - loss: 0.4035 - accuracy: 0.8129


In [3]:
dataset_gen = EDFData_GEN_TF("../Data/PSG1.edf", channels=['F4'])

In [4]:
def our_generator():
    for X, Y in dataset_gen:
        # Consistency with the other test
        X = np.squeeze(X)

        # Scale the data
        X = (X - dataset_gen.mean.item())/dataset_gen.std.item()
        X = np.expand_dims(X, -1)
        # Binarize labels
        Y = 1 if 5 else 0

        yield X,Y

In [57]:
dataset = tf.data.Dataset.from_generator(our_generator, 
                                        output_types=(tf.float64, tf.int32),
                                        output_shapes=(tf.TensorShape([15360,1]),tf.TensorShape([]))).batch(16)

In [60]:
for a, b in dataset:
    break
a.shape, b.shape

(TensorShape([16, 15360, 1]), TensorShape([16]))

In [50]:
model = create_simple_model()
history = model.fit(x=dataset, epochs=10)

Epoch 1/10
56/56 [==============================] - 9s 154ms/step - loss: 0.0188 - accuracy: 0.9932
Epoch 2/10
56/56 [==============================] - 6s 109ms/step - loss: 7.4451e-16 - accuracy: 1.0000
Epoch 3/10
56/56 [==============================] - 6s 103ms/step - loss: 6.4117e-16 - accuracy: 1.0000
Epoch 4/10
56/56 [==============================] - 5s 96ms/step - loss: 6.4083e-16 - accuracy: 1.0000
Epoch 5/10
56/56 [==============================] - 4s 69ms/step - loss: 6.4083e-16 - accuracy: 1.0000
Epoch 6/10
56/56 [==============================] - 4s 73ms/step - loss: 6.4083e-16 - accuracy: 1.0000
Epoch 7/10
56/56 [==============================] - 4s 68ms/step - loss: 6.4083e-16 - accuracy: 1.0000
Epoch 8/10
56/56 [==============================] - 4s 63ms/step - loss: 6.4083e-16 - accuracy: 1.0000
Epoch 9/10
56/56 [==============================] - 5s 91ms/step - loss: 6.4083e-16 - accuracy: 1.0000
Epoch 10/10
56/56 [==============================] - 5s 90ms/step - loss: 